# Copernicus Sentinel-2 Data Search and Exploration

This Jupyter Notebook demonstrates how to search, filter, and explore Sentinel-2 satellite imagery metadata using the `phidown` library. The workflow includes:

- **Setting Search Parameters:**  
    The notebook defines parameters such as collection name (`SENTINEL-2`), product type (`S2MSI1C`), cloud cover threshold, area of interest (AOI) in WKT format, date range, and sorting options.

- **Executing the Search:**  
    An instance of `CopernicusDataSearcher` is configured with the specified parameters and used to query the Copernicus Open Access Hub for matching Sentinel-2 products.

- **Viewing Results:**  
    The resulting metadata is loaded into a pandas DataFrame (`df`). Key columns such as `Id`, `Name`, `S3Path`, and `GeoFootprint` are displayed for quick inspection.

## Variables and Data

- **aoi_wkt:** AOI polygon in WKT format. Polygon must start and end with the same point. Coordinates must be given in EPSG 4326
- **cloud_cover_threshold:** Maximum allowed cloud cover percentage.
- **collection_name:** Data collection to search (`SENTINEL-2`).
- **columns_to_display:** List of DataFrame columns to display.
- **df:** DataFrame containing search results (1000 entries, 16 columns).
- **searcher:** Configured `CopernicusDataSearcher` instance.
- **start_date, end_date:** Date range for the search.
- **product_type, orbit_direction, order_by, top:** Additional search parameters.

In [1]:
%load_ext autoreload
%autoreload 2

# Search

You can modify the search parameters to refine your query, re-run the search, and further analyze or visualize the results as needed.

### Sentinel Products

In [2]:
from phidown.search import CopernicusDataSearcher
# You can modify these parameters as needed

aoi_wkt = "POLYGON((-95.0 29.0, -95.0 30.0, -94.0 30.0, -94.0 29.0, -95.0 29.0))"  # Define the AOI in WKT format

searcher = CopernicusDataSearcher()
# Configure the search parameters
searcher._query_by_filter(
    collection_name='SENTINEL-1',
    product_type='SLC',
    orbit_direction=None,
    cloud_cover_threshold=None,
    aoi_wkt=None,  # Example: aoi_wkt=aoi_wkt if you want to use the defined AOI
    start_date = '2023-05-03T00:00:00',
    end_date = '2024-05-03T04:00:00',
    top=1000,
    attributes={'swathIdentifier': 'S1'} # Swath identifier Stripmap: S1, ..., S6 | 'WV1,WV2' for wavemode
    # base_url defaults to 'https://catalogue.dataspace.copernicus.eu/odata/v1/Products'
    # order_by defaults to 'ContentDate/Start desc'
)

df = searcher.execute_query()
print(f"Number of results: {len(df)}")
# Display the first few rows of the DataFrame
searcher.display_results(top_n=15)

Number of results: 115


,Id,Name,S3Path,GeoFootprint,OriginDate,Attributes
0,7a99c7a6-de0c-45c9-9130-338cf8b2c01c,S1A_S1_SLC__1SDH_20240502T121147_20240502T1212...,/eodata/Sentinel-1/SAR/S1_SLC__1S/2024/05/02/S...,"{'type': 'Polygon', 'coordinates': [[[-88.3521...",2024-05-02 14:12:39,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
1,aa64afec-2a6e-4d22-b845-723394f43b47,S1A_S1_SLC__1SDV_20240430T123135_20240430T1232...,/eodata/Sentinel-1/SAR/S1_SLC__1S/2024/04/30/S...,"{'type': 'Polygon', 'coordinates': [[[-95.4496...",2024-04-30 14:41:22,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
2,d124855e-935c-42a5-bc4e-272a3bfe4df3,S1A_S1_SLC__1SDV_20240429T020729_20240429T0207...,/eodata/Sentinel-1/SAR/S1_SLC__1S/2024/04/29/S...,"{'type': 'Polygon', 'coordinates': [[[55.55252...",2024-04-29 04:02:48,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
3,4b91139d-b8a4-4fdc-9996-0150392e339c,S1A_S1_SLC__1SDH_20240423T151240_20240423T1512...,/eodata/Sentinel-1/SAR/S1_SLC__1S/2024/04/23/S...,"{'type': 'Polygon', 'coordinates': [[[46.12170...",2024-04-23 17:46:05,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
4,320e8f49-342a-4304-a5ac-8ee63b4abeb9,S1A_S1_SLC__1SDH_20240420T121147_20240420T1212...,/eodata/Sentinel-1/SAR/S1_SLC__1S/2024/04/20/S...,"{'type': 'Polygon', 'coordinates': [[[-88.3520...",2024-04-20 14:08:22,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
5,b4049e51-13af-4d6f-8d51-90eae3fffbe4,S1A_S1_SLC__1SDV_20240418T123135_20240418T1232...,/eodata/Sentinel-1/SAR/S1_SLC__1S/2024/04/18/S...,"{'type': 'Polygon', 'coordinates': [[[-95.4494...",2024-04-18 14:28:44,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
6,ef733f69-b912-4938-a2dd-c3d725d7019e,S1A_S1_SLC__1SDV_20240417T020729_20240417T0207...,/eodata/Sentinel-1/SAR/S1_SLC__1S/2024/04/17/S...,"{'type': 'Polygon', 'coordinates': [[[55.55358...",2024-04-17 04:05:12,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
7,ccd7ced8-1e71-4b99-b1dd-aa0e2e10491c,S1A_S1_SLC__1SDH_20240411T151238_20240411T1512...,/eodata/Sentinel-1/SAR/S1_SLC__1S/2024/04/11/S...,"{'type': 'Polygon', 'coordinates': [[[46.12553...",2024-04-11 17:33:37,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
8,a52cd19d-fcd7-4114-86d7-5bc0e8ecd8c8,S1A_S1_SLC__1SDH_20240408T121147_20240408T1212...,/eodata/Sentinel-1/SAR/S1_SLC__1S/2024/04/08/S...,"{'type': 'Polygon', 'coordinates': [[[-88.3524...",2024-04-08 14:10:42,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
9,2af7e445-f19c-444c-9087-6294d37c4bfb,S1A_S1_SLC__1SDV_20240406T123135_20240406T1231...,/eodata/Sentinel-1/SAR/S1_SLC__1S/2024/04/06/S...,"{'type': 'Polygon', 'coordinates': [[[-95.4495...",2024-04-06 14:45:13,"[{'@odata.type': '#OData.CSC.StringAttribute',..."


### CCM

In [ ]:
from phidown.search import CopernicusDataSearcher
# You can modify these parameters as needed

searcher = CopernicusDataSearcher()
# Configure the search parameters
searcher._query_by_filter(
    collection_name='CCM',
    product_type='NAO_MS4_2A_07B6',
    orbit_direction=None,
    cloud_cover_threshold=None,
    aoi_wkt=None,  # Use the defined AOI
    start_date='2014-05-03T00:00:00',
    end_date='2024-05-03T04:00:00',
    top=1000,
    attributes=None
)

df = searcher.execute_query()
print(f"Number of results: {len(df)}")
# Display the first few rows of the DataFrame
searcher.display_results(top_n=15)

Number of results: 1000


,Id,Name,S3Path,GeoFootprint,OriginDate,Attributes
0,58388509-8ce1-48e2-a647-45c794d0e522,SP07_NAO_MS4_2A_20221007T132334_20221007T13234...,/eodata/CCM/VHR_IMAGE_2021/NAO_MS4_2A_07B6/202...,"{'type': 'Polygon', 'coordinates': [[[-52.3474...",2024-03-05 10:05:58,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
1,d5ca94d5-65ad-427a-8a9f-47171fbca40b,SP07_NAO_MS4_2A_20221007T132334_20221007T13234...,/eodata/CCM/VHR_IMAGE_2021/NAO_MS4_2A_07B6/202...,"{'type': 'Polygon', 'coordinates': [[[-52.3115...",2024-03-05 10:06:02,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
2,833ec6d6-ae83-4987-a6cb-6e2be3ba161f,SP07_NAO_MS4_2A_20221007T132334_20221007T13234...,/eodata/CCM/VHR_IMAGE_2021/NAO_MS4_2A_07B6/202...,"{'type': 'Polygon', 'coordinates': [[[-52.6427...",2024-03-05 10:05:54,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
3,0f2bb08d-7cb5-4504-95d5-0c558518248a,SP06_NAO_MS4_2A_20221007T090438_20221007T09044...,/eodata/CCM/VHR_IMAGE_2021/NAO_MS4_2A_07B6/202...,"{'type': 'Polygon', 'coordinates': [[[23.04404...",2024-03-05 10:04:31,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
4,5a690c00-d347-4f7e-bfb2-0216e754658b,SP06_NAO_MS4_2A_20221007T090438_20221007T09044...,/eodata/CCM/VHR_IMAGE_2021/NAO_MS4_2A_07B6/202...,"{'type': 'Polygon', 'coordinates': [[[23.11499...",2024-03-05 10:04:29,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
5,fe1716a7-d75a-4f42-80ae-cac3e386b594,SP06_NAO_MS4_2A_20221007T090438_20221007T09044...,/eodata/CCM/VHR_IMAGE_2021/NAO_MS4_2A_07B6/202...,"{'type': 'Polygon', 'coordinates': [[[23.01186...",2024-03-05 10:04:31,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
6,1516455d-477e-4a75-8681-dbfccd0aa79b,SP07_NAO_MS4_2A_20221006T141714_20221006T14172...,/eodata/CCM/VHR_IMAGE_2021/NAO_MS4_2A_07B6/202...,"{'type': 'Polygon', 'coordinates': [[[-61.4695...",2024-03-05 10:05:43,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
7,b510042a-e00f-4fdb-9c6a-b2b09aa18510,SP07_NAO_MS4_2A_20221006T141714_20221006T14172...,/eodata/CCM/VHR_IMAGE_2021/NAO_MS4_2A_07B6/202...,"{'type': 'Polygon', 'coordinates': [[[-61.5506...",2024-03-05 10:05:43,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
8,28437779-59dd-44de-a94d-56051af32603,SP07_NAO_MS4_2A_20221006T141714_20221006T14172...,/eodata/CCM/VHR_IMAGE_2021/NAO_MS4_2A_07B6/202...,"{'type': 'Polygon', 'coordinates': [[[-61.5561...",2024-03-05 10:05:43,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
9,adb91962-0265-4bb1-a2ee-9630876cd9d8,SP06_NAO_MS4_2A_20221006T133215_20221006T13322...,/eodata/CCM/VHR_IMAGE_2021/NAO_MS4_2A_07B6/202...,"{'type': 'Polygon', 'coordinates': [[[-53.5266...",2024-03-05 10:06:07,"[{'@odata.type': '#OData.CSC.StringAttribute',..."


### Search by Product Name (Exact Match)

You can search for a specific product if you know its exact name.

**Tip:**

To search by exact product name, make sure the name exists in the Copernicus Data Space catalogue. You can find valid product names by running a general search first, or by browsing the [Copernicus Data Space Catalogue](https://dataspace.copernicus.eu/browser/). If your search returns no results, try using a product name from the official documentation or a recent search result.

Below, we use a known-valid product name as an example.

In [ ]:
from phidown.search import CopernicusDataSearcher

searcher_by_name = CopernicusDataSearcher()

# Replace with an actual product name you want to find
# Example product name (ensure this product exists and is accessible when testing)
# product_to_find = 'S1A_IW_GRDH_1SDV_20230503T052602_20230503T052627_048390_05D0B4_6B4C.SAFE' 
# For demonstration, let's use a placeholder. You should use a real name for actual search.
product_to_find = 'S1A_IW_GRDH_1SDV_20141031T161924_20141031T161949_003076_003856_634E.SAFE' # Example, replace with a recent, valid name

print(f"Searching for product with exact name: {product_to_find}\n")
df_exact = searcher_by_name.query_by_name(product_name=product_to_find)

if not df_exact.empty:
    searcher_by_name.display_results(top_n=1)
    display(df_exact)
else:
    print(f"Product '{product_to_find}' not found or an error occurred.")

## 📥 Download Your Product

Important:
If your credentials are not deleted automatically, you must manually revoke them here:

🔗 Manage S3 Credentials: https://eodata-s3keysmanager.dataspace.copernicus.eu/panel/s3-credentials

Stay secure — always clean up your credentials!

In [ ]:
filename = 'S1A_S3_SLC__1SDH_20240430T213606_20240430T213631_053668_0684A3_FCED.SAFE'
searcher.download_product(filename)

Alternative CLI:

In your command line 
```bash 
python -m phidown.downloader -u {username} -p {password} -eo_product_name {filename}
```


In [ ]:
from phidown.downloader import load_credentials
# Download the product using the command line
filename = 'S1A_IW_SLC__1SDV_20220503T034102_20220503T034129_043041_0523B0_16A0.SAFE'
username, password = load_credentials()
# Run the command line downloader
# Note: You need to have the phidown package installed and available in your PATH
!python -m phidown.downloader -u {username} -p {password} -eo_product_name {filename}